### Generating Benchmark Data

In [1]:
from Raster_benchmarking.generate_rasters import generate_nlm_rasters

In [2]:
generate_nlm_rasters('./Raster_benchmarking/data', layers=100)

# Running DGGS Benchmark

In [3]:
import os
from pathlib import Path
import pandas as pd
from joblib import Parallel, delayed
from Raster_benchmarking.DGGS_funcs import h3index_raster

In [4]:
discrete_dir = Path('Raster_benchmarking/data/discrete')
continuous_dir = Path('Raster_benchmarking/data/continuous')
output_dir = Path('Raster_benchmarking/data/dggs')

num_files_to_open = 100 #Change to desired input number
mode = lambda x: pd.Series.mode(x)[0]

### Indexing

##### Continuous files

In [5]:
%%time
_ = Parallel(n_jobs=-1)(delayed(h3index_raster)(file, output_dir, stem='continuous', operation=mode) for file in list(sorted(continuous_dir.glob('*.asc')))[:num_files_to_open])

CPU times: total: 172 ms
Wall time: 6.83 s


##### Discrete files

In [6]:
%%time
os.makedirs(output_dir, exist_ok=True)
_ = Parallel(n_jobs=-1)(delayed(h3index_raster)(file, output_dir, stem='discrete', operation=mode) for file in list(sorted(discrete_dir.glob('*.asc')))[:num_files_to_open])

CPU times: total: 109 ms
Wall time: 4.75 s


### Join and Classify

In [7]:
from Raster_benchmarking.DGGS_funcs import classify, summing, final_plotting

In [8]:
c_files = list(output_dir.glob('continuous*.parquet'))
d_files = list(output_dir.glob('discrete*.parquet'))

##### Continuous files

In [9]:
cont_df = classify(c_files, num_files_to_open, scale=100)
cont_df = summing(cont_df)

Joining DataFrames: 100%|██████████| 99/99 [00:00<00:00, 1903.51it/s]


In [12]:
final_plotting(cont_df)

AttributeError: 'DataFrame' object has no attribute 'h3'

##### Discrete files

In [ ]:
disc_df = classify(d_files, num_files_to_open, scale=100)
disc_df = summing(disc_df)

In [ ]:
final_plotting(disc_df)